<a href="https://colab.research.google.com/github/riyadMTRS/547/blob/main/examples/Search_grounding_for_research_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2025 Google LLC.

In [4]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Search tool with Gemini 2.0

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Search_grounding_for_research_report.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

In this tutorial you are going to leverage the latest [search tool](https://ai.google.dev/gemini-api/docs/models/gemini-v2#search-tool) of the Gemini 2.0 model to write a company report. Note that the search tool is a paid ony feature and this tutorial does not work with a free tier API key.

You may be asking, why does one need to use the search tool for this purpose? Well, as you may be aware, today's business world evolves very fast and LLMs generally are not trained frequently enough to capture the latest updates. Luckily Google search comes to the rescue. Google search is built to provide accurate and nearly realtime information and can help us fulfill this task perfectly.

Note that while Gemini 1.5 models offered the [search grounding](https://ai.google.dev/gemini-api/docs/grounding) feature which may also help you achieve similar results (see [a previous notebook](https://github.com/google-gemini/cookbook/blob/gemini-1.5-archive/examples/Search_grounding_for_research_report.ipynb) using search grounding), the new search tool in Gemini 2.0 models is much more powerful and easier to use, and should be prioritized over search grounding. It is also possible to use multiple tools together (for example, search tool and function calling).

## Setup

### Install SDK

The new **[Google Gen AI SDK](https://ai.google.dev/gemini-api/docs/sdks)** provides programmatic access to Gemini 2.0 (and previous models) using both the [Google AI for Developers](https://ai.google.dev/gemini-api/docs) and [Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/overview) APIs. With a few exceptions, code that runs on one platform will run on both. This means that you can prototype an application using the Developer API and then migrate the application to Vertex AI without rewriting your code.

More details about this new SDK on the [documentation](https://ai.google.dev/gemini-api/docs/sdks) or in the [Getting started](../quickstarts/Get_started.ipynb) notebook.

In [5]:
%pip install -U -q google-genai

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [6]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

### Import the libraries

In [7]:
from google import genai
from google.genai.types import GenerateContentConfig, Tool
from IPython.display import display, HTML, Markdown
import io
import json
import re

### Initialize SDK client

With the new SDK you now only need to initialize a client with you API key (or OAuth if using [Vertex AI](https://cloud.google.com/vertex-ai)). The model is now set in each call.

In [8]:
client = genai.Client(api_key=GOOGLE_API_KEY)

### Select a model

Search tool is a new feature in the Gemini 2.0 model that automatically retrieves accurate and grounded artifacts from the web for developers to further process. Unlike the search grounding in the Gemini 1.5 models, you do not need to set the dynamic retrieval threshold.

For more information about all Gemini models, check the [documentation](https://ai.google.dev/gemini-api/docs/models/gemini) for extended information on each of them.


In [9]:
MODEL_ID = "gemini-2.5-flash" # @param ["gemini-1.5-flash-latest","gemini-2.5-flash-lite-preview-06-17","gemini-2.5-flash","gemini-2.5-pro"] {"allow-input":true}

## Write the report with Gemini 2.0

### Select a target company to research

Next you will use Alphabet as an example research target.

In [10]:
COMPANY = 'Alphabet' # @param {type:"string"}

### Have Gemini 2.0 plan for the task and write the report

Gemini 2.0 is a huge step up from previous models since it can reason, plan, search and write in one go. In this case, you will only give Gemini 2.0 a prompt to do all of these and the model will finish your task seamlessly. You will also using output streaming, which streams the response as it is being generated, and the model will return chunks of the response as soon as they are generated. If you would like the SDK to return a response after the model completes the entire generation process,  you can use the non-streaming approach as well.

Note that you must enable Google Search Suggestions, which help users find search results corresponding to a grounded response, and [display the search queries](https://ai.google.dev/gemini-api/docs/grounding/search-suggestions#display-requirements) that are included in the grounded response's metadata. You can find [more details](https://ai.google.dev/gemini-api/terms#grounding-with-google-search) about this requirement.

In [13]:
sys_instruction = """You are an analyst that conducts company research.
You are given a company name, and you will work on a company report. You have access
to Google Search to look up company news, updates and metrics to write research reports.

When given a company name, identify key aspects to research, look up that information
and then write a concise company report.

Feel free to plan your work and talk about it, but when you start writing the report,
put a line of dashes (---) to demarcate the report itself, and say nothing else after
the report has finished.
"""

config = GenerateContentConfig(system_instruction=sys_instruction, tools=[Tool(google_search={})], temperature=0)
response_stream = client.models.generate_content_stream(
    model=MODEL_ID, config=config, contents=[COMPANY])

report = io.StringIO()
for chunk in response_stream:
  candidate = chunk.candidates[0]

  for part in candidate.content.parts:
    if part.text:
      display(Markdown(part.text))

      # Find and save the report itself.
      if m := re.search('(^|\n)-+\n(.*)$', part.text, re.M):
          # Find the starting '---' line and start saving.
          report.write(m.group(2))
      elif report.tell():
        # If there's already something in the buffer, keep recording.
        report.write(part.text)

    else:
      print(json.dumps(part.model_dump(exclude_none=True), indent=2))

  # You must enable Google Search Suggestions
  if gm := candidate.grounding_metadata:
    if sep := gm.search_entry_point:
      display(HTML(sep.rendered_content))

---
**Alphabet Inc. Company Report**

Alphabet Inc

. is an American multinational technology conglomerate holding company headquartered in Mountain View, California. Established on October 2, 2015, through a restructuring of Google, Alphabet serves as the parent company for Google and several other diverse

 subsidiaries. This restructuring aimed to enhance accountability for Google's core business and provide greater autonomy to other ventures. Alphabet is recognized as one of the world's most valuable companies

 and ranks among the top technology companies globally by revenue and profit.

**Leadership:**
Sundar Pichai serves as the Chief Executive Officer (CEO) of both Alphabet and its largest subsidiary, Google. Larry Page and Sergey Brin, the co-founders of Google, stepped down from executive roles in December 2019 but remain actively involved as employees, board members, and controlling shareholders of Alphabet. The leadership team also includes John L. Hennessy as Chairman, Ruth Porat as President and Chief Investment Officer, and Anat Ashkenazi as Chief Financial Officer.

**Business Segments and Key Products:**
Alphabet's operations are primarily divided into three segments:
*   **Google Services:** This is the largest revenue-generating segment, encompassing Google's core search and advertising businesses

, Android, Chrome, Google Maps, Google Play, and YouTube. Advertising remains the primary source of revenue for Alphabet.
*   **Google Cloud:** This segment provides enterprise-level

...

 In March 2025, Google announced an agreement to acquire cybersecurity firm Wiz.

**Market Position and Competitors:**
Alphabet is a key player among the "Big Five" American information technology companies, alongside Amazon,

 Apple, Meta, and Microsoft.
*   In **search**, Google maintains a dominant market share, competing primarily with Microsoft's Bing.
*   **Google Cloud** competes

 with major cloud service providers like Amazon Web Services (AWS) and Microsoft Azure.
*   In **hardware and smart home devices**, Alphabet competes with Apple (iPhone, HomePod) and Amazon (Echo devices).


*   Its **digital advertising** business faces competition from Amazon.com, The Trade Desk, and Criteo.
*   **YouTube** competes with streaming media platforms such as Netflix and Amazon

.com.
*   **Waymo** in the self-driving car sector competes with companies like Tesla and Uber.

Very impressive! Gemini 2.0 starts by planning for the task, performing relevant searches and streams out a report for you.

### Final output

Render the final output.

In [ ]:
display(Markdown(report.getvalue().replace('$', r'\$')))  # Escape $ signs for better MD rendering
display(HTML(sep.rendered_content))

As you can see, the Gemini 2.0 model is able to write a concise, accurate and well-structured research report for us. All the information in the report is factual and up-to-date.

Note that this tutorial is meant to showcase the capability of the new search tool and inspire interesting use cases, not to build a production research report generator. If you are looking to use a tool, please check out Google [Deep Research](https://blog.google/products/gemini/google-gemini-deep-research/) in the Gemini app.

## Next Steps

* To learn more about the search tool, check out the [Search tool documentation](https://ai.google.dev/gemini-api/docs/models/gemini-v2#search-tool).
* To get started with streaming, check out the [Streaming Quickstart](../quickstarts/Streaming.ipynb).
* To get started with the search tool, check out the [Search tool quick start](../quickstarts/Search_Grounding.ipynb).
* To see how much easier and more powerful the search tool in Gemini 2.0 is than the previous search grounding feature, compare this tutorial with the [previous example](../examples/Search_grounding_for_research_report.ipynb).

To learn more about all the Gemini features, check out the [Gemini quickstart notebooks](https://github.com/google-gemini/cookbook/tree/main/quickstarts/).